In [ ]:
import sys
import os

spark_home='/opt/spark-1.5.1-bin-hadoop2.6/'
# addin pyspark to current path
sys.path.append( spark_home+'/python' )
sys.path.append( spark_home+'/python/lib/py4j-0.8.2.1-src.zip' )

# Установка локальных переменных
os.environ["SPARK_HOME"] = spark_home
os.environ["HADOOP_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop'
os.environ["HADOOP_YARN_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop-yarn'
os.environ["YARN_CONF_DIR"] = '/etc/hadoop/conf.cloudera.yarn'
os.environ["SPARK_CLASSPATH"] = '/etc/hive/conf.cloudera.hive1'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--master yarn pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
conf = SparkConf ().set( 'spark.app.name', 'test');
sc = SparkContext (conf= conf)

# Коллаборативная фильтрация. Алгоритм ALS

<img src="images/als.PNG">

рейтинги задачются не в обычном фомате mllib, а в виде оценки для каждой пары пользователь - продукт
<table>
<tr>
<td>
**user id**
</td>
<td>
**product id**
</td>
<td>
**rating**
</td>
</tr>
<tr>
<td>
0
</td>
<td>
1
</td>
<td>
5.0
</td>
</tr>
<tr>
<td>
0
</td>
<td>
0
</td>
<td>
2.0
</td>
</tr>
<tr>
<td>
1
</td>
<td>
1
</td>
<td>
-3.0
</td>
</tr>
</table>

In [ ]:
!hadoop fs -cat /testdata/netflix_sample/ratings_sorted_by_user_sample.csv | head

In [ ]:
data = sc.textFile ("/testdata/netflix_sample")

In [ ]:
ratings = data.map (lambda line: line.split(","))\
   .filter(lambda x: x[1]!='movie_id')\
   .map (lambda fields: (int(fields[1]), int(fields[2]), float (fields[3])))

In [ ]:
ratings.take(5)

Обучение модели делается методом<br>
ALS.train(ratings,<br>
          rank, <br>
          iterations=5, <br>
          lambda_=0.01, <br>
          blocks=-1, <br>
          nonnegative=False, <br>
          seed=None)

In [ ]:
from pyspark.mllib.recommendation import ALS
rank = 20
numIterations = 10
lambda_ = 0.5
model = ALS.train (ratings, rank, numIterations, lambda_)

In [ ]:
?ALS.train (ratings, rank, numIterations)

Предсказывать оценку можно методом predictAll

In [ ]:
predictions = model.predictAll (
ratings.map (lambda x: (x[0],x[1])))

In [ ]:
predictions.take(5)

In [ ]:
predictions = predictions.map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
predictions.take(5)

In [ ]:
ratings = ratings.map (lambda r: ((r[0],r[1]),r[2]))

In [ ]:
ratings.take(5)

In [ ]:
ratings.take(5)

In [ ]:
# Evaluate the model on training data
ratesAndPreds = ratings.join(predictions)

In [ ]:
ratesAndPreds.take(5)

Среднеквадратическая ошибка:

In [ ]:
from math import sqrt
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).reduce(lambda x, y: x + y)/ratesAndPreds.count()
print "Mean Squared Error = " + str(sqrt(MSE))